# Import Required Libraries
Import the necessary libraries, including requests and pandas.

In [ ]:
# Importing the required libraries
import requests
import pandas as pd
from pyinaturalist import get_observations
import numpy as np
import json

# Send GET Request to iNaturalist API
Use the requests library to send a GET request to the iNaturalist API.

In [ ]:
laval_place_id = 27655
montreal_place_id = 187355
gatineau_place_id = 142292

# Initialize an empty DataFrame to store the observations
def get_observations_dataframe(place_id,place_name):
    df = pd.DataFrame()
    for page in range(1, 10):
        observation = get_observations(place_id=place_id, verifiable=True,per_page=200, page=str(page), quality_grade="research")
        df = df.append(observation["results"], ignore_index=True)
        df["location"] = place_name
    return df

df_naturalist = pd.DataFrame()
# Example usage
laval_place_id = 27655
data_laval = get_observations_dataframe(laval_place_id,"Laval")
data_montreal = get_observations_dataframe(montreal_place_id,"Montreal")
data_gatineau = get_observations_dataframe(gatineau_place_id,"Gatineau")

df_naturalist = pd.concat([data_laval, data_montreal, data_gatineau], ignore_index=True)

print("Data saved to 'inaturalist_data.csv'.")

In [ ]:
import pandas as pd

# Assuming df_naturalist is a pandas DataFrame
df_naturalist['observed_on'] = pd.to_datetime(df_naturalist['observed_on'], utc=True)

# Save the DataFrame to Parquet
df_naturalist.to_parquet("naturalist_sample.parquet")

In [ ]:
df_naturalist = pd.read_parquet("naturalist_sample.parquet")

In [ ]:
display(df_naturalist)

In [ ]:
import os

# Get all files in the data folder
# Read the 'sentinelle_liste_sp.csv' file from the workspace
file_path = 'sentinelle_liste_sp.csv'
df_sentinelle = pd.read_csv(file_path)
# Filter the files based on the ones that contain "sentinelle" in their name

display(df_sentinelle)
# Add any additional code here for further processing or analysis

In [ ]:
import numpy as np

def add_isInvasive_column(df_inaturalist, df_sentinelle):
    cols_to_return = list(df_inaturalist)
    cols_to_return.append("isInvasive")
    df_inaturalist['species_guess'] = df_inaturalist[['species_guess']].apply(lambda x: x.astype(str).str.lower())
    df_sentinelle["Nom_francais"] = df_sentinelle[['Nom_francais']].apply(lambda x: x.astype(str).str.lower())
    merged_df = df_inaturalist.merge(right=df_sentinelle, left_on='species_guess', right_on='Nom_francais', how='left')
    merged_df["isInvasive"] = np.where((merged_df["Nom_francais"].isnull()) | (merged_df["Nom_francais"] == np.nan), False, True)
    return merged_df[cols_to_return]

# joined_df = add_isInvasive_column(df_naturalist, df_sentinelle)
# joined_df[joined_df["isInvasive"] == True]

In [ ]:
# Keep only the 'species_guess' column and add a new column with the value 'Laval'
# Convert the values in the 'species_guess' column to lowercase and select only the 'species_guess' column
df_simplified = df_naturalist.copy()
df_simplified["observed_on"] = df_simplified["observed_on"].dt.strftime("%Y-%m-%d")
df_simplified.rename(columns={'observed_on': 'observation_date'}, inplace=True)
df_simplified['species_guess'] = df_simplified['species_guess'].astype(str).str.lower()
df_simplified = add_isInvasive_column(df_simplified, df_sentinelle)
df_simplified['longitude'] = df_simplified['geojson'].apply(lambda x: x['coordinates'][0] if 'coordinates' in x else None)
df_simplified['latitude'] = df_simplified['geojson'].apply(lambda x: x['coordinates'][1] if 'coordinates' in x else None)
df_simplified["source"] = "Community"
df_simplified = df_simplified[['species_guess', 'location', 'observation_date', 'isInvasive', 'latitude', 'longitude', 'source']]
df_simplified

# Data from government on observation


In [ ]:
gov_df = pd.read_json("especes_exo_envahissantes.json")

gov_df["species_guess"] = gov_df["properties"].apply(lambda x: x['Nom_espece_français'] if 'Nom_espece_français' in x else None)
gov_df["location"] = gov_df["properties"].apply(lambda x: x['Nom_region_administrative'] if 'Nom_region_administrative' in x else None)
gov_df["observation_date"] = gov_df["properties"].apply(lambda x: x['Date_observation'] if 'Date_observation' in x else None)

gov_df["latitude"] = gov_df["properties"].apply(lambda x: x['Latitude'] if 'Latitude' in x else None)
gov_df["longitude"] = gov_df["properties"].apply(lambda x: x['Longitude'] if 'Longitude' in x else None)
gov_df["isInvasive"] = True
gov_df["source"] = "Government"
gov_df['species_guess'] = gov_df['species_guess'].astype(str).str.lower()
gov_df = gov_df[['species_guess', 'location', 'observation_date', 'isInvasive', 'latitude', 'longitude', 'source']]
gov_df = gov_df[gov_df['location'].isin(['Montréal', 'Laval', 'Outaouais'])]
gov_df

In [ ]:
# Save the DataFrame as a JSON file with UTF-8 encoding and records orientation
union_df = pd.concat([df_simplified, gov_df], ignore_index=True)
union_df.to_json('species_with_location.json', orient='records', force_ascii=False)

# Filter Data by City
Filter the DataFrame based on the city or cities of interest.

# Save Data to CSV
Save the filtered data to a CSV file using pandas.

In [ ]:
# I want to store in a df the array in the feature field from table
df = spark.sql("SELECT explode_outer(features) FROM hive_metastore.default.animaux_precaire")

In [ ]:
display(df)